In [ ]:
# Import libraries
# SQLAlchemy relatives
# import sqlalchemy
# from sqlalchemy.ext.automap import automap_base
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func

#Config relatives
import sys
sys.path.insert(1,('../..'))#1 means staring with the jupyter notebok file path
sys.path

import tarfile
import urllib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

## Extract

In [ ]:
#Import Dataset
df=pd.read_csv('../data/external/billboard_hits.csv')

## Transform

In [ ]:
#show data columns
df.columns

In [ ]:
#drop Unnamed column
df = df.drop(['Placement', "Track", "Artist", "Year", "Month", "Day"],axis=1)
# df_clean = df.sort_values('Placement').drop_duplicates('Track', keep='last')
# df.drop_duplicates(subset=['Track'], keep='last')
# df['Decade'] = (10 * (df['Year'] // 10)).astype(str) + 's'
df

In [ ]:
df = df.drop(['Unnamed: 0'],axis=1)
df

## songs_tb

## Machine Learning

In [ ]:
# decades dataframe only take highest placement
# sixty_df = df[(df['Year'] == 1961)]
# sixty_df_clean = sixty_df.sort_values('Placement').drop_duplicates('Track', keep='last')
# sixty_df_clean
# seventy_df = df[(df['Decade'] == '1970s')]
# eighty_df = df[(df['Decade'] == '1980s')]
# ninety_df = df[(df['Decade'] == '1990s')]
# twoth_df = df[(df['Decade'] == '2000s')]
# twotn_df = df[(df['Decade'] == '2010s')]

In [ ]:
# X = sixty_df_clean[['energy']]
# y = sixty_df_clean[['loudness']]

In [ ]:
distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(df)
    distortions.append(kmeanModel.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# Create a kmeans model
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)

# Fit the model to the data
kmeans.fit(df)

# Use the data to predict the clusters
# save the predictions as `predicted_clusters`
predicted_clusters = kmeans.predict(df)
df["Cluster"] = predicted_clusters
df

In [ ]:
# Plot the clusters
plt.scatter(df['Cluster'], df['tempo'], c=df['Cluster'], s=40, cmap='viridis')
plt.show()